In [ ]:
# Solution Ex. 46 SparkSQL - V3
# The same approach of V2 but without two copies of the timestamp for each table

In [ ]:
import sys

In [ ]:
inputPath = "/data/students/bigdata-01QYD/ex_data/Ex46/data/readings.txt"
outputPath = "res_out_Ex46SparkSQLV3/"

In [ ]:
# Read the content of readings
# Create a DataFrame from persons.csv
readingsDF = spark.read.load(inputPath,\
                             format="csv",\
                             header=False,\
                             inferSchema=True)\
.withColumnRenamed("_c0","timestamp")\
.withColumnRenamed("_c1","temperature")

In [ ]:
#debug
readingsDF.show()
#readingsDF.printSchema()

In [ ]:
# Create three copies of the input data
# Each copy is associated with a different "element" of the windows
# readingsDF -> Fist elements
# readingsDFElements2 -> Second elements
# readingsDFElements3 -> Third elements
# Change only the name of columns for clearness
readingsDFElements2 = readingsDF.selectExpr("timestamp as origtimestamp2",\
                                            "temperature as temperatureElem2")

readingsDFElements3 = readingsDF.selectExpr("timestamp as origtimestamp3",\
                                            "temperature as temperatureElem3")


In [ ]:
#debug
readingsDF.show()
readingsDFElements2.show()
readingsDFElements3.show()

In [ ]:
# Join the three tables to define the windows (length 3)
# Windows with less than three elements are automatically discarded by the join 
# operation
windowsDF = readingsDF.join(readingsDFElements2, \
                            readingsDF.timestamp==(readingsDFElements2.origtimestamp2-60))\
                    .join(readingsDFElements3, \
                          readingsDFElements2.origtimestamp2==(readingsDFElements3.origtimestamp3-60))

In [ ]:
#debug
windowsDF.show()

In [ ]:
# Check increasing trend 
windowsIncreasingDF = windowsDF\
.filter("temperature<temperatureElem2 AND temperatureElem2<temperatureElem3")

In [ ]:
# Debug 
windowsIncreasingDF.show()

In [ ]:
windowsIncreasingDF.write.csv(outputPath, header=False)

In [ ]:
#!hdfs dfs -cat res_out_Ex46SparkSQLV3/part*

In [ ]:
#!hdfs dfs -rm -r res_out_Ex46SparkSQLV3